In [1]:
import sys
sys.path.append('../src/')

In [2]:
import pandas as pd
import utils

# Load opfunu functions

In [3]:
from opfunu.dimension_based import benchmark1d, benchmark2d, benchmark3d, benchmarknd
from opfunu.type_based import multi_modal, uni_modal
import inspect

In [4]:
classes = [
    benchmark1d, benchmark2d, benchmark3d, benchmarknd, # total methods = 62
    # multi_modal, uni_modal, # total methods = 47
]

In [5]:
def cls2name(cls):
    module = cls.Functions.__module__
    name = module.split('.')[-1]
    return name

def cls2methods(cls):
    return list(dict(inspect.getmembers(cls.Functions, predicate=inspect.isfunction)).keys())

def build_cls_df(classes):
    rows = []
    for cls in classes:
        name = cls2name(cls)
        methods = cls2methods(cls)
        for method_name in methods:
            clean_name = method_name.replace('_', ' ').strip()
            clean_name = ' '.join([ cn.capitalize() for cn in clean_name.split(' ') ])
            rows.append(dict(
                #cls=name,
                name=clean_name,
                #method=method_name,
                call=f'{name}.Functions().{method_name}',
            ))
    df = pd.DataFrame(rows)
    df = df.sort_values('name')
    df = df.reset_index(drop=True)
    return df

In [6]:
df = build_cls_df(classes)
df

,name,call
0,Ackley,benchmarknd.Functions()._ackley__
1,Ackley N2,benchmark2d.Functions()._ackley_n2__
2,Ackley N3,benchmark2d.Functions()._ackley_n3__
3,Ackley N4,benchmarknd.Functions()._ackley_n4__
4,Adjiman,benchmark2d.Functions()._adjiman__
...,...,...
58,Xin She Yang,benchmarknd.Functions()._xin_she_yang__
59,Xin She Yang N2,benchmarknd.Functions()._xin_she_yang_n2__
60,Xin She Yang N3,benchmarknd.Functions()._xin_she_yang_n3__
61,Xin She Yang N4,benchmarknd.Functions()._xin_she_yang_n4__


# Load Scraped Data

In [14]:
from collectors import sfu, infinity77, benchmarkfcns
import json

_s_ = ''' sfu '''
# sfu.download_homepage()
# sfu.download_functions()
sfu.crawl_functions()

_i_ = ''' infinity77 '''
# infinity77.download_homepage()
# infinity77.download_pages()
infinity77.crawl_pages()

_b_ = ''' benchmarkfcns '''
benchmarkfcns.crawl_markdown()

In [8]:
idf = infinity77.load_df()
sdf = sfu.load_df()
bdf = benchmarkfcns.load_df()

# Map opfunu benchmarks to sources

In [9]:
sources = {
    _s_: sdf,
    _i_: idf,
    _b_: bdf,
}
df = build_cls_df(classes)


adict = {}
for source_id, source_df in sources.items():
    # create mapping df
    source_map, *_ = utils.diff_map(df.name, source_df.name)
    source_map_df = pd.DataFrame(source_map.items(), columns=['name', source_id])
    # merge
    df = pd.merge(left=df, right=source_map_df, on='name', how='outer')

df

,name,call,sfu,infinity77,benchmarkfcns
0,Ackley,benchmarknd.Functions()._ackley__,Ackley,Ackley,Ackley
1,Ackley N2,benchmark2d.Functions()._ackley_n2__,NaN,NaN,Ackley N. 2
2,Ackley N3,benchmark2d.Functions()._ackley_n3__,NaN,NaN,Ackley N. 3
3,Ackley N4,benchmarknd.Functions()._ackley_n4__,NaN,NaN,Ackley N. 4
4,Adjiman,benchmark2d.Functions()._adjiman__,NaN,Adjiman,Adjiman
...,...,...,...,...,...
58,Xin She Yang,benchmarknd.Functions()._xin_she_yang__,NaN,Xin-She Yang 1,Xin-She Yang
59,Xin She Yang N2,benchmarknd.Functions()._xin_she_yang_n2__,NaN,Xin-She Yang 2,Xin-She Yang N. 2
60,Xin She Yang N3,benchmarknd.Functions()._xin_she_yang_n3__,NaN,Xin-She Yang 3,Xin-She Yang N. 3
61,Xin She Yang N4,benchmarknd.Functions()._xin_she_yang_n4__,NaN,Xin-She Yang 4,Xin-She Yang N. 4


# select which sources are superior

In [10]:
# MAYBE TODO
# rows = []
# for source_id, source_df in sources.items():
#     print(source_id)
#     row = {'source': source_id}
#     for column in source_df.columns:
#         row[column] = True
#     rows.append(row)
# pd.DataFrame(rows)

# Verify Mapping

In [11]:
# df.to_csv('df.csv', index=False)

In [12]:
# open csv file and manually remove names from sources that are wrong

In [13]:
#df = pd.read_csv('df.csv')